In [ ]:
set(y_train)

{0.0, 1.0, 2.0}

Читаем информацию о группах:

In [ ]:
group_train = []
with open("data/mq2008.train.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_train.append(int(line.split("\n")[0]))

group_valid = []
with open("data/mq2008.vali.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_valid.append(int(line.split("\n")[0]))

group_test = []
with open("data/mq2008.test.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_test.append(int(line.split("\n")[0]))

Как устроена информация о группах:
* количество групп отражает информацию о количестве запросов
* каждое число обозначает количество последовательных объектов, которые в эту группу объединяются
* из предыдущего пункта следует, что в X объекты нельзя перемешивать
* если просуммировать все числа в списке групп, получим число объектов из X

Для чего нужны группы? <br>
Для того, чтобы в обучении не сравнивать доки из разных групп (разных запросов) между собой.

In [ ]:
print(len(group_train), sum(group_train))
group_train[:10]

471 9630


[8, 8, 8, 8, 8, 16, 8, 118, 16, 8]

Обучаем модель. <br>
С помощью `eval_set` можем контролировать обучение, но это необязательный параметр, можно обучить и без валидации. <br>
В параметре `objective` можно задать три опции: `rank:ndcg`, `rank:pairwise`, `rank:map`. `ndcg` и `map` регулияруют попарный лосс с помощью подсчета соответствующих метрик.

In [ ]:
params = {'objective': 'rank:ndcg', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6, 'n_estimators': 4}

model = xgb.sklearn.XGBRanker(**params)
model.fit(x_train, y_train, group_train, verbose=True,
          eval_set=[(x_valid, y_valid)], eval_group=[group_valid])

[0]	eval_0-map:0.71552
[1]	eval_0-map:0.72606
[2]	eval_0-map:0.72795
[3]	eval_0-map:0.73352


XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=1, gamma=1.0, gpu_id=-1,
          importance_type='gain', interaction_constraints='', learning_rate=0.1,
          max_delta_step=0, max_depth=6, min_child_weight=0.1, missing=nan,
          monotone_constraints='()', n_estimators=4, n_jobs=0,
          num_parallel_tree=1, objective='rank:ndcg', random_state=0,
          reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
          tree_method='exact', validate_parameters=1, verbosity=None)

Получим предсказание на тестовом сете:

In [ ]:
pred = model.predict(x_test)

Посчитаем качество:

In [ ]:
start_idx = 0
grouped_pred = []
grouped_target = []

for group_n in group_test:
    grouped_pred.append(pred[start_idx:start_idx+group_n])
    grouped_target.append(y_test[start_idx:start_idx+group_n])
    start_idx += group_n

In [ ]:
np.mean([
    ndcg_score([grouped_target[i]], [grouped_pred[i]])
    for i in range(len(grouped_target))
])

0.5052327963105946

# Семинар и дз
Сделать и улучшить любую ML-модель на ваших проектных данных (просто клф, бленд, ранжирование, что-то что вы придумали сами...), используя любые признаки, какие захотите. Оцениваться будут:
* факт выполнения задания :)
* корректность кода (чтобы код не падал) и отсутствие логических ошибок (e.g. затестили на трейне)
* итеративность улучшения (например взяли один сет признаков, показали качество; потом добавили / подкрутили / использовали другую модель, показали качество...)
* креативность признаков
* аккуратность ноутбука

Дедлайн: 15 октября

In [ ]:
### реализуйте эту функцию ранжирования 
import collections
import string
import math
import pandas as pd
import numpy as np
import nltk

from tqdm import tqdm
from pymorphy2 import MorphAnalyzer
from razdel import tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from yargy import or_
from yargy.predicates import caseless, normalized, dictionary
from yargy import rule, and_, Parser
from yargy.predicates import gte, lte

nltk.download('stopwords')
morph = MorphAnalyzer()
stop = set(stopwords.words('russian'))


def my_preprocess(text: str):
    text = str(text)
    text = text.replace("\n", " ").replace('/', ' ')
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokenized_text = list(tokenize(text))
    lemm = [morph.parse(i.text)[0].normal_form for i in tokenized_text]
    words = [i for i in lemm if i not in stop]
#     return words
    return " ".join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Загрузим данные

In [ ]:
import io
from google.colab import files
uploaded = files.upload()

# answers_df = pd.read_excel("answers_base.xlsx")
answers_df = pd.read_excel(io.BytesIO(uploaded['answers_base.xlsx']))

# questions_df = pd.read_excel("queries_base.xlsx")
questions_df = pd.read_excel(io.BytesIO(uploaded['queries_base.xlsx']))

# df = pd.read_csv(io.StringIO(uploaded['train.csv'].decode('utf-8')))
# df

Saving answers_base.xlsx to answers_base (6).xlsx
Saving queries_base.xlsx to queries_base (6).xlsx


### Составим TF-IDF вектора и посчитаем accyracy обычного перемножения

In [ ]:
full_corpus = []

answers_df.rename(columns={'Текст вопросов': 'text', 'Номер связки': 'join_num'}, inplace=True)
questions_df.rename(columns={'Текст вопроса': 'text', 'Номер связки\n': 'join_num'}, inplace=True)

train, test_quest = train_test_split(questions_df, test_size=0.3)

train_quest = pd.concat([answers_df, train])

train_quest_processed = []
test_quest_processed = []
vectorizer = TfidfVectorizer()

# preprocess train / test
for quest in tqdm(train_quest['text']):
    train_quest_processed.append(my_preprocess(quest))
for quest in tqdm(test_quest['text']):
    test_quest_processed.append(my_preprocess(quest))
    
vectorizer.fit(train_quest_processed)
# train matrix
X_train = vectorizer.transform(train_quest_processed)
# test natrix
X_test = vectorizer.transform(test_quest_processed)

print("X_train.shape: " + str(X_train.shape))
print("X_test.shape: " + str(X_test.shape))

100%|██████████| 690/690 [00:11<00:00, 61.11it/s]


X_train.shape: (1652, 7407)
X_test.shape: (690, 7407)


In [ ]:
rating = X_train.dot(X_test.T).argmax(axis=0)
rating = np.array(rating)[0]
count = 0
for ind_test, pred in enumerate(rating):
    if math.isnan(test_quest.iloc[ind_test].join_num) or math.isnan(train_quest.iloc[pred].join_num):
        continue

    if int(test_quest.iloc[ind_test].join_num) == int(train_quest.iloc[pred].join_num):
        count += 1

"Accuracy TF-IDF: " + str(count / len(rating))

'Accuracy with natasha and TF-IDF: 0.5347826086956522'

### Попробуем удалить NER дат и поосмотрим результат после


In [ ]:
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsNERTagger,
    PER,
    NamesExtractor,
    DatesExtractor,

    Doc
)


from yargy import or_
from yargy.predicates import caseless, normalized, dictionary
from yargy import rule, and_, Parser
from yargy.predicates import gte, lte


DAY = and_(
    gte(1),
    lte(31)
)
MONTH = and_(
    gte(1),
    lte(12)
)
YEAR = and_(
    gte(1),
    lte(2018)
)
DATE = rule(
    YEAR,
    '-',
    MONTH,
    '-',
    DAY
)


MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}
MONTH_NAME = dictionary(MONTHS)
YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(normalized('год')),
    rule(normalized('число')),
    rule(caseless('числа')),
    rule(caseless('го'))
)

DATE = or_(
    rule(
        YEAR,
        '-',
        MONTH,
        '-',
        DAY
    ),
    rule(
        DAY,
        MONTH_NAME,
        YEAR,
        YEAR_WORDS.optional()
    ),
    rule(
        DAY,
        "-",
        DAY,
        MONTH_NAME,
    ),
    rule(
        DAY,
        YEAR_WORDS.optional(),
        MONTH_NAME.optional()
    )
)

parser = Parser(DATE)
text = '''
8 января 2014 года, 15 июня 2001 г.,
31 февраля 2018
Уехал 8-9 ноября в Сочи
Уезжаю 5 числа
20го сентября заболел'''
for match in parser.findall(text):
    print(match.span, [_.value for _ in match.tokens])

[1, 19) ['8', 'января', '2014', 'года']
[21, 36) ['15', 'июня', '2001', 'г', '.']
[38, 53) ['31', 'февраля', '2018']
[60, 70) ['8', '-', '9', 'ноября']
[85, 92) ['5', 'числа']
[93, 106) ['20', 'го', 'сентября']


In [ ]:
parser = Parser(DATE)
segmenter = Segmenter()
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

def preprocess_with_natasha_date(text: str) -> str:
    text = str(text)
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    for ner in doc.spans:
        text = text[0:ner.start] + text[ner.stop:]
    for match in parser.findall(text):
        for tok in match.tokens:
            text = text[0:tok.span.start] + text[tok.span.stop:]
    return " ".join(text.split())
#     return my_preprocess(text)

In [ ]:
train_quest_processed = []
test_quest_processed = []
vectorizer = TfidfVectorizer()

# preprocess train / test
for quest in tqdm(train_quest['text']):
    train_quest_processed.append(preprocess_with_natasha_date(quest))
for quest in tqdm(test_quest['text']):
    test_quest_processed.append(preprocess_with_natasha_date(quest))

vectorizer.fit(train_quest_processed)
# train matrix
X_train = vectorizer.transform(train_quest_processed)
# test natrix
X_test = vectorizer.transform(test_quest_processed)

print("X_train.shape: " + str(X_train.shape))
print("X_test.shape: " + str(X_test.shape))

100%|██████████| 690/690 [00:17<00:00, 39.40it/s]


X_train.shape: (1652, 14550)
X_test.shape: (690, 14550)


In [ ]:
rating = X_train.dot(X_test.T).argmax(axis=0)
rating = np.array(rating)[0]
count = 0
for ind_test, pred in enumerate(rating):
    if math.isnan(test_quest.iloc[ind_test].join_num) or math.isnan(train_quest.iloc[pred].join_num):
        continue

    if int(test_quest.iloc[ind_test].join_num) == int(train_quest.iloc[pred].join_num):
        count += 1

"Accuracy with yargy NER date deleting and TF-IDF: " + str(count / len(rating))

'Accuracy with natasha and TF-IDF: 0.5159420289855072'

### Результат уменьшился, тогда посмотрим как работает градиентный бустинг без удаления NER на векторах TF-IDF

In [ ]:
full_corpus = []

answers_df = answers_df.rename(columns={'Текст вопросов': 'text', 'Номер связки': 'join_num'})
answers_df = answers_df.dropna(subset=['join_num'])

questions_df = questions_df.rename(columns={'Текст вопроса': 'text', 'Номер связки\n': 'join_num'})
questions_df = questions_df.dropna(subset=['join_num'])

answers_df['normal_text'] = answers_df['text'].apply(my_preprocess)
questions_df['normal_text'] = questions_df['text'].apply(my_preprocess)


train, test_quest, y_train, y_test = train_test_split(questions_df['normal_text'],
                                                      questions_df['join_num'],
                                                      test_size=0.3)

y_train = pd.concat([answers_df['join_num'], y_train])
train_quest = pd.concat([answers_df['normal_text'], train])

train_quest_processed = []
test_quest_processed = []
vectorizer = TfidfVectorizer()

vectorizer.fit(train_quest)
# train matrix
X_train = vectorizer.transform(train_quest)
# test natrix
X_test = vectorizer.transform(test_quest)

print("X_train.shape: " + str(X_train.shape))
print("X_test.shape: " + str(X_test.shape))

X_train.shape: (1646, 7463)
X_test.shape: (687, 7463)


In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import ndcg_score

params = {'objective': 'rank:ndcg', 'learning_rate': 0.05,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6, 'n_estimators': 100}

model = xgb.sklearn.XGBClassifier(**params)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1.0,
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=0.1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
pred = model.predict(X_test)

### Градиентный бустинг показывает наилучший результат из всех моделей, которые рассматривались во всех домашних заданиях, поэтому выберем его в качестве финальной модели и найдём оптимальные параметры


In [ ]:
count = 0

for ind, i in enumerate(pred):
    if i == y_test.values[ind]:
        count += 1

print(count / len(pred))

0.6433770014556041


In [ ]:
import optuna

# 1. Define an objective function to be maximized.
def fitXGB(trial):
    params ={'objective': 'rank:ndcg',
            'max_depth':trial.suggest_int('max_depth', 3, 11),
            'n_estimators':trial.suggest_int('n_estimators', 10, 100),
            'min_child_weight':trial.suggest_int('min_child_weight', 0, 5),
            'gamma':trial.suggest_int('gamma', 0, 5),
            'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
            'nthread' : -1  
    }
    count = 0
    model = xgb.sklearn.XGBClassifier(**params)
    model.fit(X_train, y_train) 
    pred = model.predict(X_test)
    for ind, i in enumerate(pred):
        if i == y_test.values[ind]
    count += 1

    return  count / len(pred)

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(fitXGB, n_trials=10)

[I 2020-10-17 21:19:27,682] A new study created in memory with name: no-name-c466fe86-b0ed-46d7-a45c-f4361078ca7d
[I 2020-10-17 21:20:04,696] Trial 0 finished with value: 0.6215429403202329 and parameters: {'max_depth': 5, 'n_estimators': 83, 'min_child_weight': 4, 'gamma': 2, 'learning_rate': 0.09545120975343886}. Best is trial 0 with value: 0.6215429403202329.
[I 2020-10-17 21:20:57,471] Trial 1 finished with value: 0.6200873362445415 and parameters: {'max_depth': 8, 'n_estimators': 92, 'min_child_weight': 5, 'gamma': 1, 'learning_rate': 0.027233911547249193}. Best is trial 0 with value: 0.6215429403202329.
[I 2020-10-17 21:21:27,473] Trial 2 finished with value: 0.6419213973799127 and parameters: {'max_depth': 6, 'n_estimators': 56, 'min_child_weight': 3, 'gamma': 3, 'learning_rate': 0.09772431162992182}. Best is trial 2 with value: 0.6419213973799127.
[I 2020-10-17 21:22:03,649] Trial 3 finished with value: 0.5502183406113537 and parameters: {'max_depth': 8, 'n_estimators': 59, 'mi

### Посмотрим результат с наилучшими найденными параметрами

In [ ]:
params = study.best_params

In [ ]:
model = xgb.sklearn.XGBClassifier(**params)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=3,
              learning_rate=0.09772431162992182, max_delta_step=0, max_depth=6,
              min_child_weight=3, missing=None, n_estimators=56, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
pred = model.predict(X_test)
count = 0
for ind, i in enumerate(pred):
    if i == y_test.values[ind]:
        count += 1
print(count / len(pred))

0.6419213973799127
